<a href="https://colab.research.google.com/github/salmudiyanto/skripsi_online_class/blob/main/Salinan_dari_analisis_sentimen_fenomena_cpns_re_sign_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import library

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import chardet
import re
import nltk

Baca data tweet csv dan ekstrak kolom tweet


In [ ]:
with open('data master.csv', 'rb') as f:
    result = chardet.detect(f.read())

data = pd.read_csv('data master.csv', sep=';', encoding=result['encoding'])
data.head()


tweet_data = data['tweet']
tweet_akun = data['akun']
tweet_df = pd.DataFrame({'tweet': tweet_data, 'akun': tweet_akun})


symbol = '?'
tweet_df['value'] = symbol

tweet_df.to_csv('tweet.csv', index=False)

loaded_tweet_df = pd.read_csv('tweet.csv')
# print(loaded_tweet_df.head())
data

                                               tweet               akun value
0  Flas forward sepuluh tahun kemudian,saya jadi ...     <ReneSinclair>     ?
1  Lulus CPNS & Resign di Instansi ini kena denda...   <CNBC Indoneisa>     ?
2  Ratusan CPNS Resign dengan Alasan Gaji Terlalu...   <Pikiran Rakyat>     ?
3  Viral! Warganet Menyesal Jadi PNS hingga Ingin...   <Pikiran Rakyat>     ?
4   ikut tes cpns = resign baru kali ini njirr freak  <popy @inipopyyy>     ?


**1. Cleaning data dan Case Folding**

In [ ]:
with open('data_label.csv', 'rb') as f:
    result2 = chardet.detect(f.read())

data2 = pd.read_csv('data_label.csv', sep=';', encoding=result2['encoding'])
data2.head()

tweet_data = data2['Tweet']
tweet_val = data2['Value']
cleaned_tweet = []

for tweet in tweet_data:
  tweet = tweet.lower()
  tweet = re.sub(r'[^\w\s]', '', tweet)
  tweet = re.sub(r'http\S+', '', tweet)
  tweet = re.sub(r'#\w+', '', tweet)
  tweet = re.sub(r'RT|rt', '', tweet)
  tweet = re.sub(r'\bRT\b', '', tweet, flags=re.IGNORECASE)
  cleaned_tweet.append(tweet)


cleaned_tweet_df = pd.DataFrame({'tweet': cleaned_tweet, 'value': tweet_val})

cleaned_tweet_df.to_csv('tweet_clean.csv', index=False)

loaded_tweet_df = pd.read_csv('tweet_clean.csv')
# print(loaded_tweet_df.head())
loaded_tweet_df

,tweet,value
0,flas forward sepuluh tahun kemudiansaya jadi m...,Positif
1,lulus cpns resign di instansi ini kena denda ...,Negatife
2,ratusan cpns resign dengan alasan gaji terlalu...,Netral
3,viral warganet menyesal jadi pns hingga ingin ...,Netral
4,ikut tes cpns resign baru kali ini njirr freak,Negatife
...,...,...
294,terlalu banyak tingkah tuh yang re sign,Netral
295,kenapa coba mereka memilih untuk resign di lua...,Positif
296,sebaiknya pihak penyeleksi cpn sdh perhitungka...,Positif
297,mengapa hal ini harus terjadi kenapa tidak ada...,Positif


**2. Tokenizing**

In [ ]:
with open('tweet_clean.csv', 'rb') as f:
    result3 = chardet.detect(f.read())

data3 = pd.read_csv('tweet_clean.csv', sep=',', encoding=result3['encoding'])
# data3.head()

def tokenizing(tweets):
  nstr = tweets.split(' ')
  dat = []
  a = -1
  for hu in nstr:
    a = a + 1
  if hu == '':
    dat.append(a)
  p = 0
  b = 0
  for q in dat:
    b = q - p
    del nstr[b]
    p = p+1
  return nstr
data3['tweet'] = data3['tweet'].apply(tokenizing)
# data3.head(10)

token_tweet = data3
token_tweet_data = data3['tweet']
token_tweet_val = data3['value']
print(token_tweet)

# token_tweet_df = pd.DataFrame({'tweet': token_tweet_data, 'value': token_tweet_val})

# token_tweet_df.to_csv('tweet_token.csv', index=False)

# loaded_tweet_df = pd.read_csv('tweet_token.csv')
# print(loaded_tweet_df.head())

                                                 tweet     value
0    [flas, forward, sepuluh, tahun, kemudiansaya, ...   Positif
1    [lulus, cpns, , resign, di, instansi, ini, ken...  Negatife
2    [ratusan, cpns, resign, dengan, alasan, gaji, ...    Netral
3    [viral, warganet, menyesal, jadi, pns, hingga,...    Netral
4    [ikut, tes, cpns, , resign, baru, kali, ini, n...  Negatife
..                                                 ...       ...
294    [terlalu, banyak, tingkah, tuh, yang, re, sign]    Netral
295  [kenapa, coba, mereka, memilih, untuk, resign,...   Positif
296  [sebaiknya, pihak, penyeleksi, cpn, sdh, perhi...   Positif
297  [mengapa, hal, ini, harus, terjadi, kenapa, ti...   Positif
298  [please, gak, usah, pada, jadi, pns, kalo, jad...    Netral

[299 rows x 2 columns]


**3. Filtering**

In [ ]:

# nltk.download('stopwords')
# from nltk.corpus import stopwords

# with open('tweet_token.csv', 'rb') as f:
#     result4 = chardet.detect(f.read())

# data4 = pd.read_csv('tweet_token.csv', sep=',', encoding=result4['encoding'])
# # data4.head()

# def stopword_removal(tweets):
#   stop_words = set(stopwords.words('indonesian'))
#   words = tweets.split()
#   removed_words = [word for word in words if word.lower() in stop_words]
#   words = [word for word in words if word.lower() not in stop_words]
#   return ' '.join(words)
#   # return ' '.join(words), removed_words


# data4['tweet'] = data4['tweet'].apply(stopword_removal)

# filter_tweet_data = data4['tweet']
# filter_tweet_val = data4['value']

# filter_tweet_df = pd.DataFrame({'tweet': filter_tweet_data, 'value': filter_tweet_val})

# filter_tweet_df.to_csv('tweet_filter.csv', index=False)

# loaded_tweet_df = pd.read_csv('tweet_filter.csv')
# print(loaded_tweet_df.head())

In [ ]:

nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(tweets):
  filtering = stopwords.words('indonesian')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, tweets)
  for x in fit:
    data.append(x)
  return data
token_tweet['tweet'] = token_tweet['tweet'].apply(stopword_removal)
# token_tweet.head()

filter_tweet = token_tweet
filter_tweet_data = token_tweet['tweet']
filter_tweet_val = token_tweet['value']
print(filter_tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                 tweet     value
0    [flas, forward, sepuluh, kemudiansaya, manager...   Positif
1    [lulus, cpns, , resign, instansi, kena, denda,...  Negatife
2    [ratusan, cpns, resign, alasan, gaji, dianggap...    Netral
3    [viral, warganet, menyesal, pns, berhenti, ker...    Netral
4            [tes, cpns, , resign, kali, njirr, freak]  Negatife
..                                                 ...       ...
294                           [tingkah, tuh, re, sign]    Netral
295   [coba, memilih, resign, tuh, yg, pengen, pnseww]   Positif
296      [penyeleksi, cpn, sdh, perhitungkan, kendala]   Positif
297                         [antisipasi, cpns, resign]   Positif
298  [please, gak, pns, kalo, pns, itungitung, gaji...    Netral

[299 rows x 2 columns]


**4. Stamming**

In [ ]:
! pip install Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(tweets):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in tweets:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = ' '.join(do)
  print(d_clean)
  return d_clean
filter_tweet['tweet'] = filter_tweet['tweet'].apply(stemming)
stamming_tweet = filter_tweet
stamming_tweet_data = filter_tweet['tweet']
stamming_tweet_val = filter_tweet['value']
print(stamming_tweet)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.4 MB/s eta 0:00:00
flas forward puluh kemudiansaya manager rs klasik hrd rs turnover karyawan awat dominan perempuan silkus tahun resign terima cpns meni lahir cpnsresign cpns resign
lulus cpns  resign instansi kena denda rp 100 jutacpns resign cpnsresign cnbcindonesia
ratus cpns resign alas gaji anggap musibah negara cpns cpnsresign asn   lengkap youtubel5ulr3 vcwa via youtube
viral warganet sesal pns henti kerja  aku gabut sesal kerja pns  cpns cpnsresign asn
tes cpns  resign kali njirr freak
lulus pppk 2022 coba cpns tentu resign sanksi tentu khusus guru pppk cpns guru resign
opini pribadi dasar hitung ambil bijak dendan cpns resign
menpanrb tjahjo kumolo menangggapi prihal calon pegawai negeri sipil cpns pegawai perintah janji kerja pppk resign undur nyata lulus seleksi 2021
badan pegawai negara bkn catat 105 calon pegawai negeri sipil cpns undur dinayatakan lulus 2021 undur ratus cpns rugi negara vdvcnews viral asn pns

In [ ]:
# stamming_tweet_df = pd.DataFrame({'tweet': stamming_tweet_data, 'value': stamming_tweet_val})

# stamming_tweet_df.to_csv('stamming_tweet.csv', index=False)

# loaded_tweet_df = pd.read_csv('stamming_tweet.csv', encoding='latin1')
# print(loaded_tweet_df.head())

stamming_tweet.to_csv('stamming_tweet.csv', index=False)
data_stamming = pd.read_csv('stamming_tweet.csv', encoding='latin1')
data_stamming.head()


,tweet,value
0,flas forward puluh kemudiansaya manager rs kla...,Positif
1,lulus cpns resign instansi kena denda rp 100 ...,Negatife
2,ratus cpns resign alas gaji anggap musibah neg...,Netral
3,viral warganet sesal pns henti kerja aku gabu...,Netral
4,tes cpns resign kali njirr freak,Negatife


**5. Memisahkan data**

In [ ]:
data_stamming = data_stamming.astype({'value' : 'category'})
data_stamming = data_stamming.astype({'tweet' : 'string'})
data_stamming.dtypes

tweet    string[python]
value          category
dtype: object

**6. Representasi Vektor**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data_stamming['tweet'].astype('U'))
X = X_tfidf
y = data_stamming['value']
# print(X_tfidf)

vektor = pd.DataFrame({'tweet': X_tfidf, 'value': y})

vektor.to_csv('vektor.csv', index=False)

loaded_tweet_df = pd.read_csv('vektor.csv', encoding='latin1')
print(loaded_tweet_df.head())

# stamming_tweet.to_csv('stamming_tweet.csv', index=False)
# data_stamming = pd.read_csv('stamming_tweet.csv', encoding='latin1')
# data_stamming.head()

                                               tweet     value
0    (0, 142)\t0.16816090602937947\n  (0, 364)\t0...   Positif
1    (0, 131)\t0.38679474707554706\n  (0, 296)\t0...  Negatife
2    (0, 747)\t0.3207484196378678\n  (0, 728)\t0....    Netral
3    (0, 210)\t0.29779829238939226\n  (0, 31)\t0....    Netral
4    (0, 206)\t0.5347848392878656\n  (0, 463)\t0....  Negatife


In [ ]:
# Baca file Excel
file_path = 'vektor.csv'  # Ganti dengan path file Anda
df = pd.read_csv(file_path)

# Fungsi untuk memisahkan nilai berdasarkan \n
def split_column(column):
    if column.dtype == 'object':
        split_data = column.str.split('\n', expand=True)
        return split_data
    else:
        return column

# Pisahkan setiap kolom yang memiliki nilai dengan \n
split_columns = [split_column(df[col]) if df[col].dtype == 'object' else df[col] for col in df.columns]

# Gabungkan kembali hasilnya menjadi satu DataFrame
df_split = pd.concat(split_columns, axis=1)

# Simpan hasil ke file CSV baru
output_file_path = 'path_to_output_file.csv'  # Ganti dengan path file output yang Anda inginkan
df_split.to_csv(output_file_path, index=False)

In [ ]:

# Baca file CSV
file_path = 'path_to_output_file.csv'  # Ganti dengan path file Anda
df = pd.read_csv(file_path, header=None)  # Tambahkan header=None jika tidak ada baris header

# Fungsi untuk ekstraksi angka setelah titik
def extract_number(text):
    if isinstance(text, str) and '.' in text:
        # Cari posisi titik
        dot_index = text.index('.')
        # Ambil angka setelah titik
        number_after_dot = text[dot_index + 1:]
        return number_after_dot.strip()  # Hapus spasi jika ada
    else:
        return text

# Proses ekstraksi angka dari setiap sel dalam DataFrame
df_extracted = df.applymap(extract_number)

# Simpan hasil ke file CSV baru
output_file_path = 'path_to_output_file_lagi.csv'  # Ganti dengan path file output yang Anda inginkan
df_extracted.to_csv(output_file_path, index=False, header=False)  # header=False agar tidak menyimpan header

print("Data berhasil diekstrak dan disimpan ke:", output_file_path)

Data berhasil diekstrak dan disimpan ke: path_to_output_file_lagi.csv


**7. Split Data**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, data_stamming['value'], test_size=0.2, random_state=42)

data latih

In [ ]:
latih = pd.DataFrame(X_train)

latih.to_csv('data_latih.csv')
print(X_train)

  (0, 154)	0.40136444795400644
  (0, 96)	0.31239947016108227
  (0, 42)	0.40136444795400644
  (0, 258)	0.37075990765119465
  (0, 151)	0.40136444795400644
  (0, 544)	0.3591246014333766
  (0, 481)	0.37075990765119465
  (0, 138)	0.08427606591968925
  (0, 576)	0.09015296952500201
  (1, 227)	0.29709445500065884
  (1, 82)	0.2846271564662539
  (1, 149)	0.29709445500065884
  (1, 646)	0.5488812637054927
  (1, 346)	0.29709445500065884
  (1, 120)	0.29709445500065884
  (1, 303)	0.22710108154624772
  (1, 147)	0.21964052207338816
  (1, 677)	0.18837411508261243
  (1, 334)	0.2897378439030034
  (1, 384)	0.1925146165991399
  (1, 138)	0.06238208690790383
  (1, 576)	0.06673223670969167
  (2, 220)	0.24164161268150264
  (2, 722)	0.24164161268150264
  (2, 458)	0.24164161268150264
  :	:
  (236, 649)	0.14291839610100546
  (236, 619)	0.15641167733149702
  (236, 334)	0.11770114641643965
  (236, 533)	0.1021096918792513
  (236, 212)	0.15616953803252961
  (236, 138)	0.05068335600211279
  (237, 389)	0.370269335551891

data uji

In [ ]:
print(X_test)

  (0, 493)	0.392255553015951
  (0, 267)	0.3812390292933853
  (0, 353)	0.4621587929625446
  (0, 597)	0.392255553015951
  (0, 317)	0.33506995637247056
  (0, 95)	0.34118396609869567
  (0, 511)	0.29411322981295684
  (0, 568)	0.09859838838650416
  (0, 135)	0.09184011300859829
  (1, 625)	0.2510157854983525
  (1, 175)	0.2510157854983525
  (1, 461)	0.2510157854983525
  (1, 18)	0.2510157854983525
  (1, 108)	0.2510157854983525
  (1, 354)	0.2510157854983525
  (1, 38)	0.2510157854983525
  (1, 355)	0.2510157854983525
  (1, 656)	0.2077344569885781
  (1, 33)	0.2318634925275546
  (1, 639)	0.14992897600342028
  (1, 396)	0.21827472440950235
  (1, 299)	0.48095103615485646
  (1, 205)	0.15133909165790121
  (1, 544)	0.2318634925275546
  (1, 328)	0.12231927420498186
  :	:
  (58, 415)	0.33985845156289435
  (58, 125)	0.33985845156289435
  (58, 561)	0.30406906222495145
  (58, 86)	0.33985845156289435
  (58, 536)	0.33985845156289435
  (58, 632)	0.29552926205757624
  (58, 207)	0.30406906222495145
  (58, 102)	0.295

label sentimen data latih

In [ ]:
print(y_train)

6      Negatif
182    Positif
170    Positif
146    Negatif
30     Negatif
        ...   
188    Negatif
71     Negatif
106    Positif
270    Negatif
102    Negatif
Name: value, Length: 238, dtype: category
Categories (3, object): ['Negatif', 'Netral', 'Positif']


label sentimen data uji

In [ ]:
print(y_test)

159    Negatif
264    Positif
254    Positif
9      Positif
77     Negatif
234    Positif
93     Negatif
109    Negatif
5      Positif
172     Netral
97     Negatif
194    Positif
183     Netral
154    Positif
57     Positif
60     Negatif
147     Netral
108    Positif
63     Positif
140     Netral
155    Positif
104    Negatif
246    Positif
46     Negatif
42     Positif
168    Positif
279    Positif
116    Positif
226    Positif
277    Positif
17     Negatif
282     Netral
33     Positif
24     Negatif
45     Negatif
7      Negatif
113    Negatif
193    Positif
111    Negatif
92     Negatif
75     Positif
82      Netral
118    Positif
76     Negatif
129    Positif
196     Netral
209     Netral
287     Netral
186    Positif
177    Positif
144    Negatif
185     Netral
84     Negatif
247    Positif
281     Netral
73     Negatif
228     Netral
25     Positif
208     Netral
59     Positif
Name: value, dtype: category
Categories (3, object): ['Negatif', 'Netral', 'Positif']


**8. Latih K-NN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k = 3
knn_model = KNeighborsClassifier(n_neighbors=k)

knn_model.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

**9. Prediksi dan evaluasi**

In [ ]:
y_pred = knn_model.predict(X_test)
from sklearn.metrics import classification_report, accuracy_score

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

results_df = pd.DataFrame({'tweet': X_test, 'predicted_value': y_pred})

results_df.to_csv('hasil_prediksi_evaluasi.csv', index=False)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy: 0.7
Classification Report:
               precision    recall  f1-score   support

     Negatif       0.55      0.63      0.59        19
      Netral       0.77      0.77      0.77        13
     Positif       0.80      0.71      0.75        28

    accuracy                           0.70        60
   macro avg       0.70      0.71      0.70        60
weighted avg       0.71      0.70      0.70        60



**10. Implementasi**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import re
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

data_stamming = pd.read_csv('stamming_tweet.csv', encoding='latin1')
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data_stamming['tweet'].astype('U'))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, data_stamming['value'], test_size=0.2, random_state=42)
k = 7
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train)
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()
tweet_input = input("Masukkan tweet: ")
cleaned_tweet = re.sub(r'[^\w\s]', '', tweet_input)
stemmed_tweet = stemmer.stem(cleaned_tweet)
tweet_vector = tfidf_vectorizer.transform([stemmed_tweet])
predicted_sentiment = knn_model.predict(tweet_vector)[0]
# sentiment_map = {0: 'Negatif', 1: 'Netral', 2: 'Positif'}
# predicted_sentiment_label = sentiment_map[predicted_sentiment]
print("Prediksi Sentimen:", predicted_sentiment)



Masukkan tweet: Biasanya yg pinter bgt tp gaji kecil  penempatan jauh. prsh swasta sebaliknya.. Ya gitu d.
Prediksi Sentimen: Negatif


In [ ]:
# # hasil_prediksi = pd.DataFrame({
# #     "kata tweet": cleaned_tweet,  # Ganti data_uji dengan data uji yang Anda gunakan
# #     "value aktual": y_test,  # Label sentimen aktual dari data uji
# #     "value prediksi": y_pred  # Label sentimen yang diprediksi oleh model
# # })
# # hasil_prediksi.to_csv("hasil_prediksi.csv", index=False)

# # tweet_baru = input('Masukkan tweet : ')
# # print(cleaned_tweet)

# stemmer_factory = StemmerFactory()
# stemmer = stemmer_factory.create_stemmer()

# tweet_input = input("Masukkan tweet: ")

# cleaned_tweet = re.sub(r'[^\w\s]', '', tweet_input)
# stemmed_tweet = stemmer.stem(cleaned_tweet)

# tweet_vector = tfidf_vectorizer.transform([stemmed_tweet])

# predicted_sentiment = knn_model.predict(tweet_vector)[0]

# sentiment_map = {0: 'negatif', 1: 'netral', 2: 'positif'}
# predicted_sentiment_label = sentiment_map[predicted_sentiment]

# print("Prediksi Sentimen:", predicted_sentiment_label)

Masukkan tweet: apa sih ini betulan kah?


KeyError: ignored